## Instalación de bibliotecas y paquetes necesarios

In [ ]:
!pip uninstall tensorflow -y
!pip install  tensorflow==2.3.1
!pip uninstall Keras -y
!pip install Keras==2.2.4
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.19.2
!pip uninstall scikit-image -y
!pip install scikit-image==0.18.2

!pip install kraken

#paquetes necesarios en linux
! apt-get install poppler-utils
! apt install tesseract-ocr
! apt install libtesseract-dev


#bibliotecas para python
! pip install pdf2image 
! pip install pytesseract
! pip install Pillow
! pip install pytesseract